In [ ]:
from PIL import Image, ImageFilter, ImageChops
import numpy as np

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
LOAD_PATH = "nsfw.pth"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

deprocess = lambda x: x * torch.Tensor([0.229, 0.224, 0.225]).to(DEVICE) + \
        torch.Tensor([0.485, 0.456, 0.406]).to(DEVICE)

In [ ]:
model = torchvision.models.googlenet()
model.fc = nn.Linear(model.fc.in_features, 148)
model.load_state_dict(torch.load(LOAD_PATH))
model = model.to(DEVICE)
model.eval()

In [ ]:
def make_step(model, image, end, channels, iterations, step):
    x = preprocess(image).unsqueeze(0).to(DEVICE)
    x.requires_grad = True

    model.zero_grad()

    for i in range(iterations):
        y = x

        for (name, child) in model.named_children():
            y = child(y)
            if name == end:
                break

        loss = y[:, channels[0]:channels[1], :, :].norm()
        loss.backward()

        x.data = x.data + step * x.grad.data

    return x

In [ ]:
def deep_dream(model, image, end, channels, iterations, step, octave_scale, num_octave):
    if num_octave > 0:
        new_image = image.filter(ImageFilter.GaussianBlur(2))

        if(new_image.size[0] / octave_scale < 1 or new_image.size[1] / octave_scale < 1):
            size = new_image.size
        else:
            size = (int(new_image.size[0] / octave_scale), int(new_image.size[1] / octave_scale))
        new_image = new_image.resize(size, Image.ANTIALIAS)

        new_image = deep_dream(model, new_image, end, channels, iterations, step, octave_scale, num_octave - 1)

        new_image = new_image.resize(image.size, Image.ANTIALIAS)
        image = ImageChops.blend(image, new_image, 0.6)

    ret = make_step(model, image, end, channels, iterations, step)

    ret = ret.data.squeeze()
    ret = ret.transpose(0, 1)
    ret = ret.transpose(1, 2)
    ret = deprocess(ret).clamp(0, 1)

    return Image.fromarray(np.uint8(ret.cpu() * 255)).resize(image.size)

In [ ]:
image = Image.open('sky.jpg')
deep_dream(model, image, end="inception4c", channels=CHANNELS, iterations=10,
           step=1.5, octave_scale=1.4, num_octave=4)